In [1]:
import os
import time
import re
import bisect
from collections import OrderedDict
import numpy as np
import tensorflow as tf
import scipy.ndimage
import scipy.misc

import config
import misc
import tfutil
import train
import dataset

In [2]:
def generate_fake_images(labels ,run_id, snapshot=None, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir(config.result_dir, config.desc)
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/7),:] ####
        latents = np.tile(latents, [7, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:int(latents.shape[0]/6)] = [0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        labels[int(latents.shape[0]/6):int(latents.shape[0]/6)*2] = [0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002 with attractive
        labels[int(latents.shape[0]/6)*2:int(latents.shape[0]/6)*3] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[int(latents.shape[0]/6)*3:int(latents.shape[0]/6)*4] = [0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006 with black instead of brown hair
        labels[int(latents.shape[0]/6)*4:int(latents.shape[0]/6)*5] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[int(latents.shape[0]/6)*5:int(latents.shape[0]/6)*6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0] #### 000007 without young
        labels[int(latents.shape[0]/6)*6:int(latents.shape[0]/6)*7] = [1,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with bald instead of black hair
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blond hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

In [2]:
def generate_fake_images(labels ,run_id, snapshot=None, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir(config.result_dir, config.desc)
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/7),:] ####
        latents = np.tile(latents, [7, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:int(latents.shape[0]/6)] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[int(latents.shape[0]/6):int(latents.shape[0]/6)*2] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with female
        labels[int(latents.shape[0]/6)*2:int(latents.shape[0]/6)*3] = [1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with blond hair instead of black
        labels[int(latents.shape[0]/6)*3:int(latents.shape[0]/6)*4] = [1,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with bald instead of black hair
        labels[int(latents.shape[0]/6)*4:int(latents.shape[0]/6)*5] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with eyeglasses
        labels[int(latents.shape[0]/6)*5:int(latents.shape[0]/6)*6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with moustache
        labels[int(latents.shape[0]/6)*6:int(latents.shape[0]/6)*7] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0] #### 000007 without young
        labels[int(latents.shape[0]/6)*7:int(latents.shape[0]/6)*8] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1] #### 000007 with wavy hair
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blond hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

In [ ]:
def generate_fake_images(labels ,run_id, snapshot=None, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir(config.result_dir, config.desc)
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/7),:] ####
        latents = np.tile(latents, [7, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:int(latents.shape[0]/6)] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[int(latents.shape[0]/6):int(latents.shape[0]/6)*2] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with female
        labels[int(latents.shape[0]/6)*2:int(latents.shape[0]/6)*3] = [1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with blond hair instead of black
        labels[int(latents.shape[0]/6)*3:int(latents.shape[0]/6)*4] = [1,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with bald instead of black hair
        labels[int(latents.shape[0]/6)*4:int(latents.shape[0]/6)*5] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with eyeglasses
        labels[int(latents.shape[0]/6)*5:int(latents.shape[0]/6)*6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007 with moustache
        labels[int(latents.shape[0]/6)*6:int(latents.shape[0]/6)*7] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0] #### 000007 without young
        labels[int(latents.shape[0]/6)*7:int(latents.shape[0]/6)*8] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1] #### 000007 with wavy hair
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blond hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

In [3]:
run_id = 60
grid_size=[10,8];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=None, grid_size = grid_size )

Initializing TensorFlow...
Loading network from "results/060-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-006067.pkl"...
Saving results to results/062-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH
Generating png 0 / 1...
